Visto che i passi mi serviranno praticamente sempre in forma normalizzata, mi salvo anche la versione del dataset con la colonna dei passi normalizzati.
Inoltre probabilmente è meglio avere anche lo score in forma normalizzata.
Normalizzo anche le date usando minmax.

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('../data/valutati.csv.gz')

In [4]:
from sklearn.preprocessing import MinMaxScaler
import datetime

df.date = df.date.apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
df.date = MinMaxScaler().fit_transform(df[["date"]])

In [5]:
from sklearn.preprocessing import StandardScaler # Questo scaler è meno sensibile agli outliers

def scale(x):
 x["n_steps"]=StandardScaler().fit_transform(x[["steps"]]) # Lo scaling è fatto sulla base dell'utente
 x["n_sleep_score"]=StandardScaler().fit_transform(x[["sleep_score"]]) # Lo scaling è fatto sulla base dell'utente
 return x

df=df.groupby("uuid").apply(scale,include_groups=False)
df.head()

date    steps  sleep_score  \
uuid                                                                         
1003e58667235e01b49008155604980b3900b00e 0  0.335165  4309.81     0.547576   
                                         1  0.343407  2228.88     0.415384   
                                         2  0.359890  7340.69     0.734624   
                                         3  0.373626  8644.73     0.685811   
                                         4  0.420330  1643.17     0.169635   

                                             n_steps  n_sleep_score  
uuid                                                                 
1003e58667235e01b49008155604980b3900b00e 0 -0.808859      -0.571941  
                                         1 -1.256592      -1.367748  
                                         2 -0.156735       0.554103  
                                         3  0.123842       0.260243  
                                         4 -1.382613      -2.847176

In [6]:
df = df.rename_axis(["uuid","id"]).reset_index().drop(columns=["id"]) # Per un csv più pulito, infatti il groupby aggiunge una colonna che non voglio
df.head()

,uuid,date,steps,sleep_score,n_steps,n_sleep_score
0,1003e58667235e01b49008155604980b3900b00e,0.335165,4309.81,0.547576,-0.808859,-0.571941
1,1003e58667235e01b49008155604980b3900b00e,0.343407,2228.88,0.415384,-1.256592,-1.367748
2,1003e58667235e01b49008155604980b3900b00e,0.359890,7340.69,0.734624,-0.156735,0.554103
3,1003e58667235e01b49008155604980b3900b00e,0.373626,8644.73,0.685811,0.123842,0.260243
4,1003e58667235e01b49008155604980b3900b00e,0.420330,1643.17,0.169635,-1.382613,-2.847176


In [7]:
df.to_csv('../data/scalati.csv.gz',compression='gzip',index=False)